In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model

In [2]:
# returns dataframe
# params: full -> True = full dataset, False = first file

def grab_data(full, path):
    if full:
        return pd.concat([pd.read_csv(path+file, index_col=False) for file in os.listdir(path)])
    else:
        for file in os.listdir(path):
            return pd.read_csv(path+file, index_col=False)

In [3]:
# import data

nBodies = 3
workDir = "C:/Users/llave/Documents/nBody/"
dataPath = workDir + "/data/"
        
df = grab_data(True, dataPath)
df.head()

,finalFile,eventID,m1,m2,m3,x1,x2,x3,y1,y2,...,xf3,yf1,yf2,yf3,dxf1,dxf2,dxf3,dyf1,dyf2,dyf3
0,1,10001,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.442821,-3.888589,-4.899092,-2.849904,0.457428,-0.525155,0.687588,-0.222422,0.961199,-0.100621
1,1,10002,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.445500,-3.889573,-4.895165,-2.850298,0.389737,-0.420452,0.683916,-0.281080,1.049364,-0.101038
2,1,10003,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.448164,-3.890785,-4.890894,-2.850693,0.321736,-0.315285,0.680246,-0.339542,1.137235,-0.101454
3,1,10004,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.450814,-3.892225,-4.886281,-2.851090,0.253345,-0.209536,0.676576,-0.397869,1.224904,-0.101870
4,1,10005,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.453450,-3.893893,-4.881325,-2.851489,0.184487,-0.103088,0.672908,-0.456125,1.312466,-0.102286


In [4]:
# INPUT: want mass, x/y pos, dx/dy for n bodies -> total input is n*5 + 1 (time)
# OUTPUT: want x/y, dx/dy -> total input is n*4

i_col, o_col = [], []
colNames = ["m", "x", "y", "dx", "dy"]

for col in colNames:
    for n in range(1, nBodies+1):
        i_col.append(col+str(n))
        if col != "m":
            o_col.append(col+"f"+str(n))
i_col.append("t")
    
X1 = df[i_col].values
y1 = df[o_col].values

X_train,X_test,y_train,y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1147503, 16) (1147503, 12)
(286876, 16) (286876, 12)


In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [21]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d0 = Dense(128, activation='relu', input_dim=(32,16))
        self.d1 = Dense(128,activation='relu')
        self.d2 = Dense(y_train.shape[1],activation='linear')
        self.flatten = Flatten()

    def call(self, x):
        x = self.d0(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d1(x)
        x = self.d1(x)
        x = self.d1(x)
        x = self.d1(x)
        x = self.d1(x)
        x = self.d1(x)
        x = self.d1(x)
        x = self.d1(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_mse = tf.keras.metrics.MeanSquaredError(name='train_mse')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_mse = tf.keras.metrics.MeanSquaredError(name='test_mse')

In [26]:
@tf.function
def train_step(initial, path):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        prediction = model(initial, training=True)
        loss = loss_object(path, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_mse(path, prediction)
    
@tf.function
def test_step(initial, path):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(initial, training=False)
    t_loss = loss_object(path, predictions)

    test_loss(t_loss)
    test_mse(path, predictions)

In [28]:
import numpy as np

EPOCHS = 5

for epoch in range(EPOCHS):
    
    print(epoch)
    
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_mse.reset_states()
    test_loss.reset_states()
    test_mse.reset_states()

    for x, y in train_ds:
        train_step(x, y)

    for x, y in test_ds:
        test_step(x, y)

    template = 'Epoch {}, Loss: {}, MSE: {}, Test Loss: {}, Test MSE: {}'
    print(template.format(epoch + 1,
                        train_loss.result(),
                        train_mse.result() * 100,
                        test_loss.result(),
                        test_mse.result() * 100))

0
Epoch 1, Loss: 5.391846179962158, Accuracy: 539.1866455078125, Test Loss: 5.694832801818848, Test Accuracy: 569.4815063476562
1
Epoch 2, Loss: 5.147197723388672, Accuracy: 514.7137451171875, Test Loss: 7.587319374084473, Test Accuracy: 758.7301635742188
2
Epoch 3, Loss: 5.08369779586792, Accuracy: 508.37115478515625, Test Loss: 4.674722671508789, Test Accuracy: 467.4711608886719
3
Epoch 4, Loss: 5.0645222663879395, Accuracy: 506.45147705078125, Test Loss: 5.430893898010254, Test Accuracy: 543.0885009765625
4
Epoch 5, Loss: 4.957759380340576, Accuracy: 495.7795104980469, Test Loss: 5.05276346206665, Test Accuracy: 505.2748107910156
